In [1]:
import h5py 
import numpy as np 
import pandas as pd 

from matplotlib import pyplot as plt 
from sklearn.metrics import mean_squared_error as mse

import sklearn
from sklearn.model_selection import train_test_split

In [2]:
!ls ~/Downloads/vigna-2021-v4-vqtl-all-utf-v2.h5


/home/nadia/Downloads/vigna-2021-v4-vqtl-all-utf-v2.h5


In [3]:
with h5py.File("/home/nadia/Downloads/vigna-2021-v4-vqtl-all-utf-v2.h5", "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    keys = list(f.keys())
    d = {k: list(f[k]) for k in keys}
markers_v4 = pd.read_csv("/home/nadia/Downloads/markers_filtered-v4.csv", index_col='Unnamed: 0')
minicore = pd.read_csv("/home/nadia/Downloads/minicore_characterization.csv")
minicore["bVI no."] = [bytes(m.replace('\t', ''), 'utf-8') for m in minicore["VI no."]]
idx_minicore = minicore.set_index("bVI no.")
no = []

for s in d['species']:
    if s in idx_minicore.index:
        no.append(str(idx_minicore['No.'][s]))
    else:
        no.append(None)

no = np.array(no)
response = np.array([m[0] for m in d["response_R1"]])

data = pd.DataFrame(np.array([no[no != None], response[no != None]]).T
                    , columns = ['No.', 'response_R1'])
X = markers_v4.loc[data["No."]]
y = data.response_R1
X.index = np.arange(0, 1301)

Keys: <KeysViewHDF5 ['doy', 'geo_id', 'gr_covar', 'gr_names', 'month', 'response_EM', 'response_R1', 'response_R3', 'response_R5', 'response_R7', 'response_R8', 'species', 'year']>


In [4]:
import heapq
heap = []
heapq.heappush(heap, 1)
heapq.heappush(heap, 10)
heapq.heappop(heap)

class MaxHeap:
    def __init__(self, key=None):
        self.heap = []
        self.key = key
    
    def push(self, elem):
        if self.key is None:
            heapq.heappush(self.heap, -elem)
        else:
            heapq.heappush(self.heap, (-self.key(elem), elem))
        
    def pop(self):
        if self.key is None:
            return -heapq.heappop(self.heap)
        return heapq.heappop(self.heap)[1]
        
    def empty(self):
        return len(self.heap) == 0
    
    
heap = MaxHeap()
heap.push(1)
heap.push(8)
heap.push(100)
heap.pop(), heap.pop(), heap.pop()

(100, 8, 1)

In [5]:
heap= []
heapq.heappush(heap, (1, []))
heapq.heappush(heap, (23, None))
heapq.heappop(heap)

(1, [])

In [6]:
class Leaf:
    def __init__(self, row_idx, col_idx, splitter, height=0):
        self.row_idx = row_idx
        self.col_idx = col_idx
        self.splitter = splitter
        
        self.height = height
        self.value = None
        self.split_col_idx = None
        self.criterion_value = None
        self.child_row_idx_0 = None
        self.child_row_idx_1 = None
        self.child_row_idx_2 = None 
        
    def fit(self, X, y):
        self.split_col_idx, self.criterion_value = self.splitter(X, y, self.row_idx, self.col_idx)
        
        split_values = X.loc[self.row_idx, self.split_col_idx]
        
        self.child_row_idx_0 = self.row_idx[split_values == 0]
        self.child_row_idx_1 = self.row_idx[split_values == 1]
        self.child_row_idx_2 = self.row_idx[split_values == 2]
        
        self.value = np.mean(y[self.row_idx])
        return self    
        
    def predict(self, X):
        return self.value
    
    def children_idx(self):
        return self.child_row_idx_0, self.child_row_idx_1, self.child_row_idx_2
        
    def split_idx(self):
        return self.split_col_idx

In [50]:
class Tree:
    def __init__(self, row_idx, col_idx, splitter
                 , max_height, min_impurity_decrease, min_split_size):
        self.row_idx = row_idx
        self.col_idx = col_idx
        self.splitter = splitter
        
        self.max_height = max_height
        self.min_impurity_decrease = min_impurity_decrease
        self.min_split_size = min_split_size 
        
        self.nodes = []
        self.parents = []
        self.children = []
        
        
        
    def fit(self, X, y):
        #self.splitter.fit(X, y)
        self.build_tree(X, y)
        return self

    def predict(self, X):
        pass
    
    
    def can_be_splitted(self, parent_idx, child0, child1, child2):
        # if child is empty ????
        
        parent = self.nodes[parent_idx]
        N, N0, N1, N2 = len(parent.row_idx), len(child0.row_idx), len(child1.row_idx), len(child2.row_idx)
        
        if N == 0:
            return False
        
        K0, K1, K2 = N0 / N, N1 / N, N2 / N
        R = parent.criterion_value  
        R0, R1, R2 = child0.criterion_value * K0, child1.criterion_value * K1, child2.criterion_value * K2
        impurity_decrease =  R - R0 - R1 - R2 
        
        
        
        if self.max_height is not None and parent.height >= self.max_height:
            return False
        
        if self.min_impurity_decrease is not None and impurity_decrease < self.min_impurity_decrease:
            return False
        
        if self.min_split_size is not None and N < self.min_split_size:
            return False
        
        return True
            
    
    def append_leaf(self, leaf, parent_idx = -1):
        new_index = len(self.nodes)
        self.nodes.append(leaf)
        self.parents.append(parent_idx)
        self.children.append([])
        if parent_idx != -1:
            self.children[parent_idx].append(new_index)
        return new_index
        
        
    def build_tree(self, X, y):
        # appending fitted leaf!
        print("-")
        
        self.append_leaf(Leaf(self.row_idx, self.col_idx, self.splitter).fit(X, y))
        heap = MaxHeap(key=lambda leaf_idx: self.nodes[leaf_idx].criterion_value)
        heap.push(0)
    
        while not heap.empty():
            print("-")
            leaf_idx = heap.pop()
            i0, i1, i2 = self.nodes[leaf_idx].children_idx()

            
            ch0 = Leaf(i0, self.col_idx, self.splitter, self.nodes[leaf_idx].height + 1).fit(X, y)
            ch1 = Leaf(i1, self.col_idx, self.splitter, self.nodes[leaf_idx].height + 1).fit(X, y)
            ch2 = Leaf(i2, self.col_idx, self.splitter, self.nodes[leaf_idx].height + 1).fit(X, y)

            
            if self.can_be_splitted(leaf_idx, ch0, ch1, ch2):
                ch0_idx = self.append_leaf(ch0, leaf_idx)
                ch1_idx = self.append_leaf(ch1, leaf_idx)
                ch2_idx = self.append_leaf(ch2, leaf_idx)
                
                heap.push(ch0_idx)
                heap.push(ch1_idx)
                heap.push(ch2_idx)
            
                

In [8]:
class Splitter:
    def __init__(self, row_idx, col_idx):
        self.G = None
        self.row_idx = row_idx
        self.col_idx = col_idx
        
    
    def fit(X, y):
        self.G = np.cov(X)
        return self
        
        
    

In [9]:
from scipy.stats import multivariate_normal 
multivariate_normal([1, 2], [[1, 2], [2, 5]]).pdf([1, 2])

0.15915494309189535

In [10]:
np.diag(np.ones(3))

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
[x for x in X]

['NC_028351.1_1130369_G_T',
 'NC_028351.1_2589691_G_A',
 'NC_028351.1_4389407_C_T',
 'NC_028351.1_6653772_G_A',
 'NC_028351.1_7158567_A_G',
 'NC_028351.1_7158573_G_T',
 'NC_028351.1_7923697_C_T',
 'NC_028351.1_9481105_C_T',
 'NC_028351.1_10101288_G_A',
 'NC_028351.1_10609109_T_C',
 'NC_028351.1_10692107_C_G',
 'NC_028351.1_16697057_G_A',
 'NC_028351.1_22007106_G_C',
 'NC_028351.1_25184857_G_A',
 'NC_028351.1_27159258_A_G',
 'NC_028351.1_28618382_T_C',
 'NC_028351.1_34854418_G_A',
 'NC_028352.1_400760_G_A',
 'NC_028352.1_830476_G_C',
 'NC_028352.1_1084989_T_G',
 'NC_028352.1_1085005_C_T',
 'NC_028352.1_1355349_A_G',
 'NC_028352.1_1756685_A_T',
 'NC_028352.1_1765002_C_A',
 'NC_028352.1_2239136_G_C',
 'NC_028352.1_2452307_C_T',
 'NC_028352.1_4467878_A_G',
 'NC_028352.1_4467885_T_C',
 'NC_028352.1_4467886_G_T',
 'NC_028352.1_4467888_G_A',
 'NC_028352.1_4732899_A_G',
 'NC_028352.1_4864980_C_T',
 'NC_028352.1_4900994_T_G',
 'NC_028352.1_6043214_T_C',
 'NC_028352.1_10450882_A_T',
 'NC_028352.

In [12]:
def get_optim_function(lambdas, y_centered):
    n = len(lambdas)
    lambdas = np.array(lambdas)
    y_centered = np.array(y_centered)
    def func(params):
        sigma, delta = params 
        v1 = n * np.log(sigma)
        v2 = np.sum(np.log(lambdas + delta))
        v3 = 1 / sigma * np.sum(y_centered ** 2 / (lambdas + delta))
        return  v1 + v2 + v3
    return func

In [15]:
from sklearn.linear_model import LinearRegression as LR
from scipy.optimize import minimize, LinearConstraint  
from scipy.stats import multivariate_normal as mv_normal 

In [19]:
def get_optim_function_2(G, X_col, y):
    n = len(G)
    y = np.ravel(y)
    X_col = np.ravel(X_col)
    
    def func(params):
        bias, coef, sigma, delta = params
        distr = mv_normal(bias + coef * X_col, sigma * (G + delta * np.eye(n)))
        return -distr.logpdf(y)
    
    return func

In [41]:


def pdf(xj, y, G, b, bj, sg, s):
    eye = np.diag(np.ones_like(xj))
    #print(np.array(b + bj * xj))
    #print(sg * (G + s * eye))
    distr = multivariate_normal(b + bj * xj, sg * (G + s * eye))
    print(distr.pdf(y))
    return distr.pdf(y)
    
    
    

def criterion(X, y, row_idx, col_idx):
    G = np.cov(X.loc[row_idx, col_idx])
    I = np.eye(len(row_idx))
    
    y = np.array(y)
    Rs = []
    for col in col_idx: 
        X_col = np.array(X.loc[row_idx, [col]])
        opt = get_optim_function_2(G, X_col, y[row_idx])
        
        lb = [-np.inf, -np.inf, 0, 0]
        ub = [np.inf] * 4
        constraint = LinearConstraint(A = np.eye(4), lb = lb,  ub = ub)

        b, a, s, d = minimize(opt, [1, 1, 1, 1], tol = 0.01, constraints = (constraint)).x
        #print(b, a, s, d)
        #print(b + a * X_col)
        distr = mv_normal(mean = b + a * np.ravel(X_col), cov = s * (G + d * I))
        Rs.append(distr.pdf(y[row_idx]))    
    return Rs
    

def splitter(X, y, row_idx, col_idx):
    estims = criterion(X, y, row_idx, col_idx)
    
    amax = np.argmax(estims)
    split_idx = col_idx[amax]
    
    return split_idx, estims[amax]

In [30]:
M = np.cov(X.loc[row_idx, :])
w, v = np.linalg.eig(M) 
w = np.array(w, dtype='float')
v = np.array(v, dtype='float')
#np.round(v.T @ M @ v, 2)





In [44]:
col_idx = X.columns
row_idx = np.arange(40,50)
criterion(X, y, row_idx, col_idx) 

[3.437607024160801e-24,
 3.2038461913582606e-24,
 3.45093750281794e-24,
 5.58702966434968e-24,
 3.343275374029053e-24,
 3.1970788134926885e-24,
 3.2026099035318887e-24,
 3.1238501833725718e-24,
 3.3564708988604476e-30,
 4.467600429235037e-24,
 5.474807325932711e-24,
 4.5078446110745495e-24,
 3.220929293926358e-24,
 4.355891880625334e-24,
 4.751308119513534e-24,
 4.7272257087266675e-24,
 3.120813197385603e-24,
 1.2318025754597838e-24,
 1.076234231716015e-32,
 3.209465589826677e-25,
 3.209465589826677e-25,
 8.85278180598456e-24,
 4.2581155431550855e-24,
 4.2581155431550855e-24,
 3.242901670014238e-24,
 3.242901670014238e-24,
 2.3429655666673686e-33,
 2.3429655666673686e-33,
 2.3429655666673686e-33,
 2.3429655666673686e-33,
 3.441795649100688e-24,
 3.7155983208247525e-24,
 3.7155983208247525e-24,
 3.399887773438615e-24,
 3.448577700145234e-30,
 5.203148938814545e-25,
 4.7272257087266675e-24,
 3.2309296870624045e-24,
 3.2252996498123502e-24,
 3.2252996498123502e-24,
 4.7272257087266675e-24

In [51]:
rows = np.array([i for i in X.index])
cols = np.random.choice(np.array(X.columns), 20) 

tree = Tree(row_idx = rows
     , col_idx = cols
     , splitter = splitter
     , max_height = 5
     , min_impurity_decrease = 0.01
     , min_split_size = 10)

tree.fit(X, y)

-


KeyboardInterrupt: 

In [138]:
tree.nodes

In [139]:
tree.parents

[-1,
 0,
 0,
 0,
 2,
 2,
 2,
 5,
 5,
 5,
 1,
 1,
 1,
 10,
 10,
 10,
 15,
 15,
 15,
 6,
 6,
 6,
 20,
 20,
 20,
 24,
 24,
 24,
 23,
 23,
 23]

In [48]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])
a[[1], [1, 2]]

array([5, 6])

In [17]:
np.random.randint(1, 3)

1